下面提供一個以 AsyncSSH 為核心的範例，示範如何用 Python 同時當作 SSH Server 與 Client，並在 Client 執行完工作後，經由 SSH 將訊息傳到 Server，上面再呼叫 Windows 內建的 msg 命令通知使用者。

### 一、安裝套件
```bash
pip install asyncssh
```

### 二、產生 Server Host Key
> 注意：Server 需要一組金鑰讓客戶端驗證身分。
```bash
ssh-keygen -t rsa -b 2048 -f host_key
# 會產生：host_key (私鑰) 以及 host_key.pub (公鑰)
```

### 三、Server 端（server.py）

In [ ]:
# server.py
import asyncio
import asyncssh
import subprocess
import sys


class NotifySSHServerSession(asyncssh.SSHServerSession):
    def __init__(self):
        self._buffer = ""

    def data_received(self, data, datatype):
        # 收到 client 傳來的訊息
        self._buffer += data

    def eof_received(self):
        # 收到結尾，執行 Windows MSG 通知
        # 注意：需在 Server 主機上有權限執行 msg
        subprocess.call(["msg", "*", self._buffer.strip()])
        return True


async def start_server():
    await asyncssh.create_server(
        # Session handler
        lambda: NotifySSHServerSession(),
        "",  # listen on all interfaces
        8022,  # port 可自訂
        server_host_keys=["host_key"],  # 你的 host key 檔案
        # 以下可加上更嚴格的認證機制
        username="your_user",
        password="your_password",
    )


if __name__ == "__main__":
    try:
        loop = asyncio.get_event_loop()
        loop.run_until_complete(start_server())
        print("SSH Server ready on port 8022")
        loop.run_forever()
    except (OSError, asyncssh.Error) as exc:
        sys.exit(f"Error starting SSH server: {exc}")


- 重點說明
    - NotifySSHServerSession.data_received：收取來自 Client 的所有文字訊息。
    - eof_received：當接收到 EOF 時，自動呼叫 msg * <訊息>，將文字掛到 Windows 訊息系統上。
    - 你可以在 create_server 裡，改成使用公私鑰對（client_public_keys），或是更複雜的認證機制。

### 四、Client 端（client.py）

In [ ]:
# client.py
import asyncio
import asyncssh
import sys


async def notify_server(host, port, username, password, message):
    async with asyncssh.connect(
        host,
        port=port,
        username=username,
        password=password,
        known_hosts=None,  # 若不想驗 host key，可先這樣；正式環境請改成正確驗證
    ) as conn:
        # 直接將 message 發到 server 端 Session
        chan, session = await conn.create_session(
            asyncssh.SSHClientSession, term_type=None
        )
        chan.write(message)
        chan.write_eof()  # 通知 server 已傳送完畢


if __name__ == "__main__":
    if len(sys.argv) < 6:
        print('Usage: python client.py <host> <port> <user> <pass> <"your message">')
        sys.exit(1)

    _, host, port, user, pwd, msg = sys.argv
    asyncio.get_event_loop().run_until_complete(
        notify_server(host, int(port), user, pwd, msg)
    )
    print("訊息已發送給 Server")


使用方式

```bash
python client.py 192.168.1.100 8022 your_user your_password "工作已完成！"
```

### 五、完整流程

1. Server：
   - 啟動 `python server.py`，在指定埠（預設 8022）監聽 SSH 要求。
   - 確保防火牆開放該埠，且 Windows 能執行 `msg`。
2. Client：
   - 完成某項工作後，呼叫 `python client.py … "你的通知文字"`，透過 SSH 把文字傳到 Server。
   - Server 收到 EOF 後，直接執行 `msg * <文字>`，桌面上就會跳出訊息通知。

---
這套架構簡單、易擴充，你也可以改用 Paramiko 或將認證改成金鑰模式、加入 TLS 隧道、或在訊息中夾帶檔案做更複雜的資料傳遞。若有其他問題，隨時再討論！